# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114204e+02     1.545482e+00
 * time: 0.38082098960876465
     1     1.062450e+01     8.735415e-01
 * time: 2.324432849884033
     2    -1.150322e+01     1.066261e+00
 * time: 3.0440988540649414
     3    -3.414880e+01     7.389272e-01
 * time: 4.134299039840698
     4    -4.754985e+01     5.257573e-01
 * time: 5.191606044769287
     5    -5.707745e+01     2.302140e-01
 * time: 6.235673904418945
     6    -5.997113e+01     1.587956e-01
 * time: 6.940937042236328
     7    -6.105442e+01     5.053277e-02
 * time: 7.6262969970703125
     8    -6.145664e+01     7.273477e-02
 * time: 8.319392919540405
     9    -6.172694e+01     4.493600e-02
 * time: 9.009606838226318
    10    -6.192414e+01     2.957156e-02
 * time: 9.70747685432434
    11    -6.206414e+01     1.842176e-02
 * time: 10.399018049240112
    12    -6.211739e+01     1.803401e-02
 * time: 11.089184045791626
    13    -6.216723e+01     1.303048e-02
 * time: 11.782397985458374
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557719
    AtomicNonlocal      14.8522684
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336822

    total               -62.261666457076
